In [4]:
# Import all the modules
import pandas as pd

In [5]:
# Load the dataset
df = pd.DataFrame()
df = pd.read_csv("tox21_with_smiles.csv")

In [7]:
df.head()

,Unnamed: 0,index,ID,inchikey,sdftitle,order,set,CVfold,NR.AhR,NR.AR,...,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53,smiles
0,0,0,NCGC00178831-03,PEJLNXHANOHNSU-UHFFFAOYSA-N,NCGC00178831-03,0,training,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.Nc1ccc2cc3ccc(N...
1,1,1,NCGC00166114-03,SEACYXSIPDVVMV-UHFFFAOYSA-L,NCGC00166114-03,1,training,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...
2,2,2,NCGC00263563-01,WQGJEAMPBSZCIF-VEKNOCPUSA-N,NCGC00263563-01,2,training,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,CO[C@@H]1[C@@H](OC)[C@H](C)[C@@](O)(CC(=O)[O-]...
3,3,3,NCGC00013058-02,CNYGFPPAGUCRIC-UHFFFAOYSA-L,NCGC00013058-02,3,training,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,CN(C)c1ccc(C(=C2C=CC(=[N+](C)C)C=C2)c2ccccc2)c...
4,4,4,NCGC00167516-01,LYCYLGFSIXIXAB-NUZRHMIVSA-N,NCGC00167516-01,4,training,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(=O)O.CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=...


In [8]:
# Test with dataset
print(df.shape)

(11759, 21)


In [ ]:
# Add new features
# Molecular weight
# Count the occurrences of each amino acid
# Count the total number of atoms, including carbon (C), hydrogen (H), oxygen (O), nitrogen (N), sulfur (S), etc.
# Count the number of aromatic rings
# logP
# ph, charge?
# number of hydrogen, covalent bond, diff bonds


In [ ]:
# How to assess toxicity??

# NR.AhR: Can indicate systemic toxicity and cancer risk.
# NR.AR and NR.AR.LBD: Related to endocrine disruption and reproductive toxicity.
# NR.Aromatase: Can indicate hormonal imbalances and reproductive effects.
# NR.ER and NR.ER.LBD: Associated with endocrine disruption and risk of hormone-related cancers.
# NR.PPAR.gamma: Involved in metabolic toxicity.
# SR.ARE: Marker for oxidative stress and organ-specific toxicity.
# SR.ATAD5: Indicates DNA damage and potential carcinogenicity.
# SR.HSE: Associated with cellular stress and cytotoxicity.
# SR.MMP: Related to tissue damage and organ dysfunction.
# SR.p53: Strong indicator of genotoxicity and cancer risk.

In [3]:
# Preprocess the dataset

In [4]:
# Split the dataset 

In [ ]:
# Build the model

In [ ]:
# Test the accuracy of the set